In [1]:
import urllib.request
import json
import dml
import prov.model
import datetime
import uuid
import statistics
from bson.code import Code


contributor = 'anuragp1_jl101995'
reads = ['anuragp1_jl101995.subway_stations,' 'anuragp1_jl101995.turnstile']
writes = ['anuragp1_jl101995.turnstile_coordinates']


# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate('anuragp1_jl101995', 'anuragp1_jl101995')

subwaydata = repo.anuragp1_jl101995.subway_stations.find()
turnstiledata = repo.anuragp1_jl101995.turnstile.find()

repo.dropPermanent('turnstile_coordinates')
repo.createPermanent('turnstile_coordinates')

map_function_subway = Code('''function() {
                            emit(this.name, this.name);
                             }
                            ''')

map_function_turnstile = Code('''function() {
                            emit(this.STATION, this.STATION);
                             }
                            ''')

reduce_function = Code('''function(key, values) {
                               var names = [];
                               for (var i = 0; i < values.length; i++) {
                                if(!(names.indexOf(values[i]) >= 0)) {
                               names.push(values[i]);
                               }
                            }
                                    var names_obj = {};
                                    for (var i = 0; i < names.length; ++i){
                                        names_obj[i] = names[i];
                                    }
                                 return names_obj;
                               }''')

# use map reduce to get names for subway_stations
station_result = repo.anuragp1_jl101995.subway_stations.map_reduce(map_function_subway, reduce_function, 'anuragp1_jl101995.turnstile_coordinates')
station_names = []
for doc in station_result.find():
        station_names.append(doc['_id'])

# print(station_names)
# for name in station_names:
# 	print(name)

turnstile_result = repo.anuragp1_jl101995.turnstile.map_reduce(map_function_turnstile, reduce_function, 'anuragp1_jl101995.turnstile_coordinates')
turnstile_names = []
for doc in turnstile_result.find():
    turnstile_names.append(doc['_id'])

print(str(len(set(station_names))))
print(str(len(set(turnstile_names))))


#print(turnstile_names)
# print('\n')
# for name in turnstile_names:
# 	print(name)

# get matches
matched_pairs = [] # tuples of matches 
subway_matches, turnstile_matches = set(), set() # all matched names  

for station in station_names:
    this_station = station.lower()
    this_station = this_station.replace('rd', '')
    this_station = this_station.replace('th', '')
    this_station = this_station.replace('st', '')
    this_station = this_station.replace('e', '')

    for stile in turnstile_names:
        this_stile = stile.lower()
        this_stile = this_stile.replace('rd', '')
        this_stile = this_stile.replace('th', '')
        this_stile = this_stile.replace('st', '')
        this_stile = this_stile.replace('e', '')

        if this_stile == this_station:
            matched_pairs.append((station, stile))
            subway_matches.add(station)
            turnstile_matches.add(stile)
            #station_names.remove(station)
            #turnstile_names.remove(stile)
            break


# print(matched_pairs)
# unmatched_subway = set(station_names) - subway_matches
# unmatched_turnstile = set(turnstile_names) - turnstile_matches
# print('\n')
# print(unmatched_subway)
# print('\n')
# print(unmatched_turnstile)






360
568


In [2]:
# matched_pairs = [] # tuples of matches 
# subway_matches, turnstile_matches = set(), set() # all matched names  

# for station in station_names:
#     this_station = station.lower()
#     this_station = this_station.replace('rd', '')
#     this_station = this_station.replace('th', '')
#     this_station = this_station.replace('st', '')
#     this_station = this_station.replace('e', '')

#     for stile in turnstile_names:
#         this_stile = stile.lower()
#         this_stile = this_stile.replace('rd', '')
#         this_stile = this_stile.replace('th', '')
#         this_stile = this_stile.replace('st', '')
#         this_stile = this_stile.replace('e', '')

#         if this_stile == this_station:
#             matched_pairs.append((station, stile))
#             subway_matches.add(station)
#             turnstile_matches.add(stile)
#             #station_names.remove(station)
#             #turnstile_names.remove(stile)
#             break


In [3]:
manual_matches = {('Jamaica - 179th St', 'JAMAICA 179 ST'),('Kingston - Throop Aves', 'KINGSTON-THROOP'),
                  ('149th St - Grand Concourse', '149/GRAND CONC'),('Nereid Ave (238 St)', 'NEREID AV'),('Smith - 9th Sts', 'SMITH-9 ST'),
                  ('Whitehall St', 'WHITEHALL S-FRY'),('Delancey St', 'DELANCEY/ESSEX'),('66th St - Lincoln Ctr', '66 ST-LINCOLN'),
                  ('Essex St', 'DELANCEY/ESSEX'),('42nd St - Port Authority Bus Term','42 ST-PORT AUTH'), ('8th St - NYU', '8 ST-NYU'),
                  ('Roosevelt Island - Main St', 'ROOSEVELT ISLND'),('3rd Ave - 138th St', '3 AV 138 ST'),('Lower East Side - 2nd Ave', '2 AV'),
                  ('161st St - Yankee Stadium', '161/YANKEE STAD'),('W 4th St - Washington Sq (Upper)', 'W 4 ST-WASH SQ'),('4th Ave', '4 AV-9 ST'), 
                  ('Jackson Hts - Roosevelt Av', '82 ST-JACKSON H'),('Lexington Ave - 53rd St', 'LEXINGTON AV/53'),('34th St - Penn Station', '34 ST-PENN STA'),
                  ('Bleecker St (Downtown)', 'BLEECKER ST'), ('Central Park North (110th St)', 'CENTRAL PK N110'),('Grand Ave - Newtown', 'GRAND-NEWTOWN'),
                  ('Howard Beach - JFK Airport', 'HOWARD BCH JFK'),('Vernon Blvd - Jackson Ave', 'VERNON-JACKSON'),('Lexington Ave - 63rd St', 'LEXINGTON AV/63'),
        ('Forest Hills - 71st Av', 'FOREST HILLS 71'),('Woodside - 61st St', '61 ST WOODSIDE'),('Flushing - Main St', 'FLUSHING-MAIN'),
        ('Jamaica Ctr - Parsons / Archer','JAMAICA CENTER'), ('Ave P', 'AVENUE P'),('15th St - Prospect Park', '15 ST-PROSPECT'), 
        ('Briarwood - Van Wyck Blvd', 'BRIARWOOD'),('72nd St', '72 ST'),('Aqueduct - North Conduit Av', 'AQUEDUCT N.COND'),
        ('Wyckoff Ave', 'MYRTLE-WYCKOFF'),('Broadway - Nassau St', 'NASSAU ST'),('63rd Dr - Rego Park', '63 DR-REGO PARK'),
        ('5th Ave - Bryant Pk', '5 AVE'),('Park Pl', 'PARK PLACE'),('Broadway Junction', 'BROADWAY JCT'),('Coney Island - Stillwell Av', 'CONEY IS-STILLW'),
        ('Broadway - Lafayette St', "B'WAY-LAFAYETTE"),('Eastchester - Dyre Ave', 'EASTCHSTER/DYRE'),('Hoyt - Schermerhorn Sts', 'HOYT-SCHER'),
        ('Times Sq - 42nd St', 'TIMES SQ-42 ST'),('Mets - Willets Point', 'METS-WILLETS PT'),('Norwood - 205th St', 'NORWOOD 205 ST'),
        ('Bay Ridge - 95th St', 'BAY RIDGE-95 ST'),('W 4th St - Washington Sq (Lower)'), ('W 4 ST-WASH SQ'),('40th St', '40 ST LOWERY ST'),
        ('68th St - Hunter College', '68ST-HUNTER CO'),('138th St - Grand Concourse', '138/GRAND CONC'),('137th St - City College', '137 ST CITY COL'),
        ('Queens Plz', 'QUEENS PLAZA'),('75th St - Eldert Ln', '75 ST-ELDERTS'),('59th St - Columbus Circle', '59 ST COLUMBUS'),
        ('Bushwick - Aberdeen', 'BUSHWICK AV'),('Grand Army Plaza', 'GRAND ARMY PLAZ'),('Brooklyn College - Flatbush Ave', 'FLATBUSH AV-B.C'),
        ('Crown Hts - Utica Ave', 'CROWN HTS-UTICA'),('Brooklyn Bridge - City Hall', 'BROOKLYN BRIDGE'), ('West Farms Sq - E Tremont Av', 'WEST FARMS SQ'),
        ('Kew Gardens - Union Tpke', 'KEW GARDENS'),('9th St', '4 AV-9 ST'),('Ave M', 'AVENUE M'),('Far Rockaway - Mott Ave', 'FAR ROCKAWAY'),
        ('Marble Hill - 225th St', 'MARBLE HILL-225'),('52nd St', '52 ST'),('Wakefield - 241st St', 'WAKEFIELD/241'),('42nd St - Bryant Pk', '42 ST-BRYANT PK'),
        ('Clinton - Washington Aves', 'CLINTON-WASH AV'),('Ave X', 'AVENUE X'),('163rd St - Amsterdam Av', '163 ST-AMSTERDM'),
        ('Sutter Ave - Rutland Road', 'SUTTER AV-RUTLD'),('Christopher St - Sheridan Sq', 'CHRISTOPHER ST'),('Astoria - Ditmars Blvd', 'ASTORIA DITMARS'),
        ('Atlantic Av - Pacific St', 'ATL AV-BARCLAY'),('Long Island City - Court Sq', 'COURT SQ'),
        ('Ave J', 'AVENUE J'),('Harlem - 148 St', 'HARLEM 148 ST'),('3rd Ave - 149th St', '3 AV-149 ST'),('Ave N', 'AVENUE N'),('Knickerbocker Ave', 'KNICKERBOCKER'),
        ('Eastern Pkwy - Bklyn Museum', 'EASTN PKWY-MUSM'),('Grand Central - 42nd St', 'GRD CNTRL-42 ST'),('Cortlandt St (NB only)', 'CORTLANDT ST'),
        ('Cortlandt St (Temporarily Closed)', 'CORTLANDT ST'), ('Westchester Sq - E Tremont Ave', 'WESTCHESTER SQ'),('Van Cortlandt Park - 242nd St', 'V.CORTLANDT PK'),
        ('Ave H', 'AVENUE H'),('82nd St - Jackson Hts', '82 ST-JACKSON H'),('E 105th St', 'EAST 105 ST'),('Jay St - Borough Hall', 'JAY ST-METROTEC'),
        ('Bedford - Nostrand Aves', 'BEDFORD-NOSTRAN'),('Bedford Park Blvd', 'BEDFORD PK BLVD'),('116th St - Columbia University', '116 ST-COLUMBIA'),
        ('Canarsie - Rockaway Pkwy', 'CANARSIE-ROCKAW'),('81st St', '81 ST-MUSEUM'),('103rd St - Corona Plaza', '103 ST-CORONA'),('Herald Sq - 34th St', '34 ST-HERALD SQ'),
        ('Myrtle-Willoughby Aves', 'MYRTLE-WILLOUGH'),('5th Ave - 53rd St', '5 AV/53 ST'),('Bay Pky', 'BAY PKWY'),('Union Sq - 14th St', '14 ST-UNION SQ'),
        ('Inwood - 207th St', 'INWOOD-207 ST'),('Ozone Park - Lefferts Blvd', 'OZONE PK LEFFRT'),('Ft Hamilton Pkwy', 'FT HAMILTON PKY'),('Ave I', 'AVENUE I'),
        ('Cathedral Pkwy (110th St)', 'CATHEDRAL PKWY'),('Rockaway Park - Beach 116 St', 'ROCKAWAY PARK B'),('Ave U', 'AVENUE U'),('47th-50th Sts - Rockefeller Ctr', '47-50 STS ROCK'),
        ('W 8th St - NY Aquarium', 'W 8 ST-AQUARIUM'),('Sutphin Blvd - Archer Av', 'SUTPHIN-ARCHER'),('74th St - Broadway', '74 ST-BROADWAY'),
        ('21st St - Queensbridge', '21 ST-QNSBRIDGE'),('World Trade Center', 'WORLD TRADE CTR'),('182nd-183rd Sts', '182-183 STS'),('5th Ave - 59th St', '5 AV/59 ST'),
        ("E 143rd St - St Mary's St", "E 143/ST MARY'S"),('Jamaica - Van Wyck', 'JAMAICA VAN WK'),('90th St - Elmhurst Av', '90 ST-ELMHURST'),
        ('Hunters Point Ave', 'HUNTERS PT AV')}
print("PRINTING PAIRS ")
print("========================")
print(matched_pairs)
print("========================")
print("========================")
print('PRINTING MANUAL')
print(manual_matches)
print("========================")


all_matches = set(list(matched_pairs) + list(manual_matches))
print(all_matches)

PRINTING PAIRS 
[('103rd St', '103 ST'), ('104th St', '104 ST'), ('110th St', '110 ST'), ('111th St', '111 ST'), ('116th St', '116 ST'), ('121st St', '121 ST'), ('125th St', '125 ST'), ('135th St', '135 ST'), ('145th St', '145 ST'), ('14th St', '14 ST'), ('155th St', '155 ST'), ('157th St', '157 ST'), ('167th St', '167 ST'), ('168th St', '168 ST'), ('169th St', '169 ST'), ('170th St', '170 ST'), ('174th St', '174 ST'), ('174th-175th Sts', '174-175 STS'), ('175th St', '175 ST'), ('176th St', '176 ST'), ('181st St', '181 ST'), ('183rd St', '183 ST'), ('18th Ave', '18 AV'), ('18th St', '18 ST'), ('190th St', '190 ST'), ('191st St', '191 ST'), ('1st Ave', '1 AV'), ('207th St', '207 ST'), ('20th Ave', '20 AV'), ('215th St', '215 ST'), ('219th St', '219 ST'), ('21st St', '21 ST'), ('225th St', '225 ST'), ('231st St', '231 ST'), ('233rd St', '233 ST'), ('238th St', '238 ST'), ('23rd St', '23 ST'), ('25th Ave', '25 AV'), ('25th St', '25 ST'), ('28th St', '28 ST'), ('30th Ave', '30 AV'), ('33rd

In [4]:
import urllib.request
import json
import dml
import prov.model
import datetime
import uuid
import statistics
from bson.code import Code

print('starting')
contributor = 'anuragp1_jl101995'
reads = ['anuragp1_jl101995.subway_stations,' 'anuragp1_jl101995.turnstile']
writes = ['anuragp1_jl101995.turnstile_coordinates']


# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate('anuragp1_jl101995', 'anuragp1_jl101995')


turnstile_data = repo.anuragp1_jl101995.turnstile.find()

repo.dropPermanent('turnstile_bydate')
repo.createPermanent('turnstile_bydate')


import pandas as pd 

ts_data = []
for entry in turnstile_data:
    ts_data.append([entry['STATION'], entry['DATE'], entry['ENTRIES']])
df = pd.DataFrame(ts_data, columns = ['STATION', 'DATE', 'ENTRIES'])
print(df)
    
    
print('finished')

starting
                STATION        DATE  ENTRIES
0                 59 ST  09/24/2016  5838478
1                 59 ST  09/24/2016  5838515
2                 59 ST  09/24/2016  5838530
3                 59 ST  09/24/2016  5838635
4                 59 ST  09/24/2016  5838951
5                 59 ST  09/24/2016  5839333
6                 59 ST  09/25/2016  5839510
7                 59 ST  09/25/2016  5839541
8                 59 ST  09/25/2016  5839561
9                 59 ST  09/25/2016  5839659
10                59 ST  09/25/2016  5839863
11                59 ST  09/25/2016  5840123
12                59 ST  09/26/2016  5840224
13                59 ST  09/26/2016  5840240
14                59 ST  09/26/2016  5840307
15                59 ST  09/26/2016  5840490
16                59 ST  09/26/2016  5840833
17                59 ST  09/26/2016  5841712
18                59 ST  09/27/2016  5841926
19                59 ST  09/27/2016  5841952
20                59 ST  09/27/2016  5842003
2

In [5]:
print('starting')
df_max = df
df_max = df_max.groupby(by=['STATION', 'DATE'], as_index=False)['ENTRIES'].max()
#df_max
print('finished df_max')
df_min = df
df_min = df_min.groupby(by=['STATION', 'DATE'], as_index=False)['ENTRIES'].min()
#df_min
print('finished df_min')
df_min['ENTRIES'] *= -1
print('finished df_min_max')
#df_min


starting
finished df_max
finished df_min
finished df_min_max


In [6]:
temp_df1 = df_max
temp_arr1 = []
for index, row in temp_df1.iterrows():
    row_1 = row[1]
    row_1 = row_1.replace('/', '')
    row_1 = row_1[-4:] + row_1[:-4]
    temp_arr1.append([row[0],int(row_1),row[2]])
    
temp_df1 = pd.DataFrame(temp_arr1, columns = ['STATION', 'DATE', 'ENTRIES'])
temp_df1 = temp_df1.sort(['STATION','DATE'], ascending=[False,True])
temp_df1



//anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,STATION,DATE,ENTRIES
243192,ZEREGA AVE,20150419,1237821
243193,ZEREGA AVE,20150420,1237821
243194,ZEREGA AVE,20150421,1237821
243195,ZEREGA AVE,20150422,1237821
243196,ZEREGA AVE,20150423,1237823
243197,ZEREGA AVE,20150424,1237824
243198,ZEREGA AVE,20150425,1237824
243199,ZEREGA AVE,20150426,1237825
243200,ZEREGA AVE,20150427,1237970
243201,ZEREGA AVE,20150428,1238147


In [7]:
temp_df1[10000:11000]


,STATION,DATE,ENTRIES
233437,VERNON-JACKSON,20160514,11360860
233438,VERNON-JACKSON,20160515,11362643
233439,VERNON-JACKSON,20160516,11365657
233440,VERNON-JACKSON,20160517,11368914
233441,VERNON-JACKSON,20160518,11372267
233442,VERNON-JACKSON,20160519,11375526
233443,VERNON-JACKSON,20160520,11378731
233444,VERNON-JACKSON,20160521,11380756
233445,VERNON-JACKSON,20160522,11382419
233446,VERNON-JACKSON,20160523,11385470


In [8]:
temp_df = temp_df1
last_station = ''
last_entry = 0
temp_arr = []
last_nonzero = ['place', 0]
for index, row in temp_df.iterrows():
    
    if row[0] == last_station:
        temp =row[2] 
        row_2 = abs(row[2] - last_entry) 
        
        if row_2 == 0:
            if last_nonzero[0] == row[0]:
                row_2 = abs(row[2] - last_nonzero[1])
        else:
            last_nonzero = [row[0], row[2]]     
        if row_2 < 200000 and row_2 != 0:            
            temp_arr.append([row[0], row[1], row_2])
        last_entry = temp  
    
    else:
        temp = row[2]
        row_2 = 0
        last_entry = temp     
        
    last_station = row[0]
    
temp_df = pd.DataFrame(temp_arr, columns = ['STATION', 'DATE', 'ENTRIES'])
temp_df


,STATION,DATE,ENTRIES
0,ZEREGA AVE,20150423,2
1,ZEREGA AVE,20150424,1
2,ZEREGA AVE,20150426,1
3,ZEREGA AVE,20150427,145
4,ZEREGA AVE,20150428,177
5,ZEREGA AVE,20150429,201
6,ZEREGA AVE,20150430,228
7,ZEREGA AVE,20150501,240
8,ZEREGA AVE,20150502,82
9,ZEREGA AVE,20150503,80


In [9]:
temp_df.describe()


,DATE,ENTRIES
count,2.222350e+05,222235.000000
mean,2.015468e+07,1496.924364
std,4.966950e+03,1680.941504
min,2.014123e+07,1.000000
25%,2.015060e+07,693.000000
50%,2.015111e+07,1331.000000
75%,2.016042e+07,2049.000000
max,2.016093e+07,188991.000000


In [ ]:
count = 0
for index, row in temp_df.iterrows():
    if row[2] == 0:
        count += 1
print(count)

0


In [27]:
# print('Starting')
# client = dml.pymongo.MongoClient()
# repo = client.repo
# repo.authenticate('anuragp1_jl101995', 'anuragp1_jl101995')

# weatherdata = repo.anuragp1_jl101995.weather.find()
# date_weather = []

# repo.dropPermanent('turnstile_weather')
# repo.createPermanent('turnstile_weather')
    
# for w in weatherdata:
#     if int(w['DATE']) >= 20150423:
#         for index, row in temp_df.iterrows():
#             #print('w is ' + str(int(w['DATE'])) + ' and row[1] is ' + str(row[1]))
#             if int(w['DATE']) == row[1]:
#                 #print('MATCH: w is ' + str(w) + ' and row[1] is ' + str(row[1]))
#                 datestring = str(w['DATE'])[4:6] + '/' + str(w['DATE'])[6:8] + '/' + str(w['DATE'])[0:4]
#                 avgtemp = statistics.mean([w['TMAX'], w['TMIN']])
#                 insert_weather = {'Date': datestring, 'AvgTemp': avgtemp, 'Precip': w['PRCP'], 
#                                   'Station' : row[0], 'Entries' : row[2]}
#                 repo.anuragp1_jl101995.turnstile_weather.insert_one(insert_weather)
            
# print('Finished')

In [29]:
# Add daily totals for each turnstile to the mongoDB
repo.dropPermanent('turnstile_total_byday')
repo.createPermanent('turnstile_total_byday')

for entry in temp_arr:
    insert_ts_byday = {'STATION': entry[0], 'DATE': entry[1], 'ENTRIES' : entry[2]}
    repo.anuragp1_jl101995.turnstile_total_byday.insert_one(insert_ts_byday)

temp_df = pd.DataFrame(temp_df.groupby(by=['DATE'])['ENTRIES'].sum())
temp_df

,ENTRIES
DATE,
20141228,305733
20141229,517749
20141230,546054
20141231,514681
20150101,304701
20150102,471797
20150103,342883
20150104,281275
20150105,590578


In [30]:
weatherdata = repo.anuragp1_jl101995.weather.find()
date_weather = []

# Collection for associating daily weather with each turnstile entry
repo.dropPermanent('turnstile_weather')
repo.createPermanent('turnstile_weather')

print('Comparing weatherdata and temp_df') ######

for w in weatherdata:
    for index, row in temp_df.iterrows():
    #print('w is ' + str(int(w['DATE'])) + ' and row[1] is ' + str(row[1]))
        if int(w['DATE']) == index:
        #print('MATCH: w is ' + str(w) + ' and row[1] is ' + str(row[1]))
            datestring = str(w['DATE'])[4:6] + '/' + str(w['DATE'])[6:8] + '/' + str(w['DATE'])[0:4]
            avgtemp = statistics.mean([w['TMAX'], w['TMIN']])
            insert_weather = {'Date': datestring, 'AvgTemp': avgtemp, 'Precip': w['PRCP'], 'Entries' : int(row[0])}
            repo.anuragp1_jl101995.turnstile_weather.insert_one(insert_weather)

            break
print('Finished')

Comparing weatherdata and temp_df
Finished


In [2]:
import urllib.request
import json
import dml
import prov.model
import datetime
import uuid
import statistics
from bson.code import Code


contributor = 'anuragp1_jl101995'
# reads = ['anuragp1_jl101995.subway_stations,' 'anuragp1_jl101995.turnstile']
# writes = ['anuragp1_jl101995.turnstile_coordinates']

# Set up the database connection.
client = dml.pymongo.MongoClient()
repo = client.repo
repo.authenticate('anuragp1_jl101995', 'anuragp1_jl101995')




print('Starting')
citibike_data = repo.anuragp1_jl101995.citibike.find()
# "starttime" : "2014-01-01 00:16:13"
cb_data = []
for entry in citibike_data:
    date = entry['starttime']
    date = date[:date.index(' ')]
    if '-' in date:
        date = date.split('-')
    elif '/' in date:
        date = date.split('/') 
    # 5/31/2015 12:12:12
    # date = ['5', '31, '2015']
    # [2015 - 4 - 10]
    
    if len(date[0]) < 2:
        date[0] = '0' + date[0]
    if len(date[1]) < 2:
        date[1] = '0' + date[1]
    if len(date[2]) < 2:
        date[2] = '0' + date[2]
        
    if date[0] in ['2014', '2015', '2016']:
        new_date = date[0] + date[1] + date[2]
        cb_data.append([new_date, entry['start station name']])
        
    elif date[2] in ['2014', '2015', '2016']:
        new_date = date[2] + date[0] + date[1]
        cb_data.append([new_date, entry['start station name']])


print('Finished')

Starting
Finished


In [4]:
import pandas as pd
print('Starting')

cb_df = pd.DataFrame(cb_data, columns = ['date', 'start_station_name'])
cb_df

Starting


,date,start_station_name
0,20140101,Catherine St & Monroe St
1,20140101,1 Ave & E 30 St
2,20140101,E 48 St & 3 Ave
3,20140101,Pershing Square N
4,20140101,Atlantic Ave & Fort Greene Pl
5,20140101,W 59 St & 10 Ave
6,20140101,E 47 St & 1 Ave
7,20140101,W 4 St & 7 Ave S
8,20140101,Division St & Bowery
9,20140101,Lexington Ave & E 26 St


In [5]:
print('Starting')

cb_df['Count'] = cb_df['start_station_name']
counted_cb_df = pd.DataFrame(cb_df.groupby(['date'], as_index=False)['Count'].count())
counted_cb_df

Starting


,date,Count
0,20140101,6059
1,20140102,8600
2,20140103,1144
3,20140104,2292
4,20140105,2678
5,20140106,9510
6,20140107,6267
7,20140108,9246
8,20140109,13354
9,20140110,9847


In [6]:
weatherdata = repo.anuragp1_jl101995.weather.find()
date_weather = []

# Collection for associating daily weather with each turnstile entry
repo.dropPermanent('citibike_weather')
repo.createPermanent('citibike_weather')

print('Comparing weatherdata and counted_cb_df') ######

for w in weatherdata:
    for index, row in counted_cb_df.iterrows():
        #print('weather: ' + str(w['DATE']))
        #print('Citibike: ' + str(row[0]))
        
        if int(float(w['DATE'])) == int(row[0]):
            #print('MATCH: w is ' + str(w) + ' and row[1] is ' + str(row[1]))
            datestring = str(w['DATE'])[4:6] + '/' + str(w['DATE'])[6:8] + '/' + str(w['DATE'])[0:4]
            avgtemp = statistics.mean([w['TMAX'], w['TMIN']])
            insert_weather = {'Date': datestring, 'AvgTemp': avgtemp, 'Precip': w['PRCP'], 'Citibike_Usage' : int(row[1])}
            repo.anuragp1_jl101995.citibike_weather.insert_one(insert_weather)

            break
print('Finished')

Comparing weatherdata and counted_cb_df
Finished
